In [ ]:
import pandas as pd
import os
import sys
import random
import seaborn as sns
import numpy as np
sys.path.insert(0, '../../')
sys.path.insert(0, '../../cycif/')
from common_apis import *

In [ ]:
os.chdir('Z:/sorger/data/IN Cell Analyzer 6000/Connor/Fixed MCF10 Common/20x full exp/20180905_Updated/')
plate_info = pd.read_csv('d:/data/MCF10A 090718 data/plate_key.csv',index_col=0)
channel_info = pd.read_csv('d:/data/MCF10A 090718 data/channel_key.csv',index_col=0)
valid_plates = list(range(1,19))
plate_info.index = plate_info.index.astype(str)
channel_info.index = channel_info.index.astype(str)
print(plate_info.head())
print(channel_info.head())

In [ ]:
metadata = pd.DataFrame()
pooled_data = pd.DataFrame()
data_type = 'Mean'
for plate in valid_plates:
    plate_id = plate
    plate = 'Plate' + str(plate)
    print(plate)
    # Assumes the data was organized by plate and results are in the /analysis folder
    path_analysis = os.path.join(plate,'analysis')
    indiv_txts = [x for x in os.listdir(path_analysis) if 'txt' in x]
    df_plate = pd.DataFrame()
    for txt in indiv_txts:
        well = txt.split('_')[1]
        field = txt.split('_')[2]
        fn = os.path.join(path_analysis,txt)
        df = pd.read_table(fn)
        cols_mean_intensity = [x for x in df.columns if (data_type+'Intensity') in x]
        df = df[cols_mean_intensity]
        sep = ''.join(['Channel',data_type,'Intensity'])
        cols = [x.split(sep )[0] + '_' + channel_info.loc[x.split(sep )[1],'Marker'] for x in df.columns]
        df.columns = cols
        df['well'] = well
        df['field'] = field
        df.index = df.apply(lambda x: '_'.join(['cell',x.well,x.field,str(x.name)]),axis = 1)
        df_plate = df_plate.append(df)
    # QC based on DAPI signalling
    lost_cells = []
    for i in range(1,8):      
        lost_cells += df_plate.index[df_plate.iloc[:,i]<=df_plate.iloc[:,i].quantile(0.03)].tolist()
    df_plate.drop(lost_cells,inplace=True)
    
    plate_id = plate[5:]
    plate_metadata = df_plate.merge(plate_info.loc[plate_id],left_on='well',right_on='Well',how='left').loc[:,'well':]
    plate_metadata['plate'] = plate_id
    metadata = metadata.append(plate_metadata)
    pooled_data = pooled_data.append(df_plate)
    
os.chdir('d:/data/MCF10A 090718 data/')
output_preix = ' '.join(['MCF10A 090718 pooled',data_type])
metadata.loc[metadata['Concentration (ng/ml)'] == 't0_ctrl','Concentration (ng/ml)'] = 0
metadata = metadata.iloc[:,2:]
metadata.columns = ['Well','ligand','dose','time','Plate']
pooled_bgnd = pd.DataFrame()
for i in range(8):
    for j in range(9):
        if (i==0 or i==4) & (j==2):
            break
        bgnd = pd.DataFrame(pooled_data.iloc[:,j+9*i]-pooled_data.iloc[:,9*i], columns=[pooled_data.columns[j+9*i]])
        pooled_bgnd = pd.concat([pooled_bgnd,bgnd],axis = 1)
valid_cols = [x for x in pooled_bgnd.columns if ('bckgrnd' not in x) & ('empty' not in x)]
pooled_bgnd = pooled_bgnd[valid_cols]
pooled_bgnd[pooled_bgnd<0] = 0
pooled_bgnd = np.log2(pooled_bgnd+1)

for key,value in {'t0_ctrl':'ctrl','control':'PBS', 'TGFb':'TGFB', 'IFNg':'IFNG'}.items():
    metadata.loc[metadata.ligand == key,'ligand'] = value
    
metadata.Plate = metadata.Plate.astype(int)
metadata['condition'] = metadata.ligand + '_' + metadata.time.astype(str)
metadata['replicate'] = 'C'
metadata.loc[metadata.Plate<=12,'replicate'] = 'B'
metadata.loc[metadata.Plate<=6,'replicate'] = 'A'
metadata['collection'] = 'C3'
metadata['sampleid'] = metadata[['ligand', 'time', 'collection', 'replicate']].astype(str).apply(lambda x: '_'.join(x),axis=1)
metadata.to_csv(output_preix+' metadata.csv')
pooled_data.iloc[:,:-2].to_csv(output_preix+' raw data.csv')
pooled_bgnd.to_csv(output_preix+' normalized data.csv')

In [ ]:
cells = []
for datatype in ['Mean','Median']:
    output_preix = ' ' .join(['MCF10A 090718 pooled',datatype])
    metadata = pd.read_csv(output_preix+' metadata.csv',index_col=0)
    metadata.index = (metadata.Plate.astype(str) + '_' + metadata.index).values
    cells.append(metadata.index.tolist())
valid_cells = list(set(cells[0])&set(cells[1]))

In [ ]:
for datatype in ['Mean','Median']:
    output_preix = ' '.join(['MCF10A 090718 pooled',datatype])
    metadata = pd.read_csv(output_preix+' metadata.csv',index_col=0)
    raw_data = pd.read_csv(output_preix+' raw data.csv',index_col=0)
    norm_data = pd.read_csv(output_preix+' normalized data.csv',index_col=0)
    
    for key,value in {'t0_ctrl':'ctrl','control':'PBS', 'TGFb':'TGFB', 'IFNg':'IFNG'}.items():
        metadata.loc[metadata.ligand == key,'ligand'] = value
    metadata.columns = ['Well', 'ligand', 'dose', 'time', 'Plate', 'condition', 'replicate','collection', 'sampleid']
    metadata.Plate = metadata.Plate.astype(int)
    metadata['condition'] = metadata.ligand + '_' + metadata.time.astype(str)
    metadata['replicate'] = 'C'
    metadata.loc[metadata.Plate<=12,'replicate'] = 'B'
    metadata.loc[metadata.Plate<=6,'replicate'] = 'A'
    metadata['collection'] = 'C3'
    metadata['sampleid'] = metadata[['ligand', 'time', 'collection', 'replicate']].astype(str).apply(lambda x: '_'.join(x),axis=1)
    
    metadata.index = (metadata.Plate.astype(str) + '_' + metadata.index).values
    raw_data.index = metadata.index
    norm_data.index = metadata.index
    filenames = [output_preix+x for x in [' metadata.csv', ' raw data.csv', ' normalized data.csv']]
    for df,fn in zip([metadata,raw_data,norm_data], filenames):
        df = df[df.index.isin(valid_cells)]
        df.to_csv(fn)